In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

names = [
    "sys_time", "license_number", "lng", "lat", "gps_time", "EMPTY1", "speed",
    "direction", "car_status", "alarm_status", "EMPTY2", "EMPTY3",
    "license_color", "recorder_speed", "mileage", "height", "EMPTY4"
]

def notify(msg):
    import datetime
    channel="J0budaR2THarZw0OqS5O"
    notify_url=f"https://notify.run/{channel}"
    massage=f"{str(datetime.datetime.now())} | {msg}"
    os.system(f'curl {notify_url} -d "{massage}"')

%pwd

'/home/cseadmin/dz/TrafficFlowModel/data_process/taxi_clean'

In [2]:
# 获取taxi中所有文件，之后读取使用
root_path = os.walk("../../data/taxi")
all_files = []
for root,ds,fs in root_path:
    days = []
    for f in fs:
        full_path = os.path.join(root,f)
        days.append(full_path)
    if len(days)==0:
        continue
    all_files.append(days)
all_files

[['../../data/taxi/202006/2020-06-04/2020-06-04_6',
  '../../data/taxi/202006/2020-06-04/2020-06-04_0',
  '../../data/taxi/202006/2020-06-04/2020-06-04_7',
  '../../data/taxi/202006/2020-06-04/2020-06-04_2',
  '../../data/taxi/202006/2020-06-04/2020-06-04_8',
  '../../data/taxi/202006/2020-06-04/2020-06-04_5',
  '../../data/taxi/202006/2020-06-04/2020-06-04_4',
  '../../data/taxi/202006/2020-06-04/2020-06-04_9',
  '../../data/taxi/202006/2020-06-04/2020-06-04_1',
  '../../data/taxi/202006/2020-06-04/2020-06-04_3'],
 ['../../data/taxi/202006/2020-06-06/2020-06-06_6',
  '../../data/taxi/202006/2020-06-06/2020-06-06_8',
  '../../data/taxi/202006/2020-06-06/2020-06-06_2',
  '../../data/taxi/202006/2020-06-06/2020-06-06_7',
  '../../data/taxi/202006/2020-06-06/2020-06-06_3',
  '../../data/taxi/202006/2020-06-06/2020-06-06_9',
  '../../data/taxi/202006/2020-06-06/2020-06-06_5',
  '../../data/taxi/202006/2020-06-06/2020-06-06_0',
  '../../data/taxi/202006/2020-06-06/2020-06-06_4',
  '../../da

data check

---

In [ ]:
df_temp=pd.read_csv("../../data/taxi/202006/2020-06-01/2020-06-01_0", names=names)

df_temp

In [ ]:
df_temp.sort_values(["license_number", "gps_time"], inplace=True)

df_temp[:100]

In [ ]:
last_row=df_temp.iloc[0]
index_list=[]
for index, row in df_temp.iterrows():
    if row.speed==0 and last_row.speed==0 and row.license_number==last_row.license_number and (row.lat!=last_row.lat or row.lng!=last_row.lng):
        index_list.append(index)
    last_row=row
    
    if len(index_list)>20:
        break

df_temp.loc[index_list]

In [ ]:
df_temp.loc[(df_temp["lng"]<=0) | (df_temp["lat"]<=0) | (df_temp["speed"]==0)]

In [ ]:
df_temp.drop_duplicates(["lat", "lng"], inplace=True)

df_temp

In [5]:
df_temp["license_number"].nunique()

22547

In [4]:
all_rows=0
dirty_rows=0
for file in sum(all_files, []): # all_files.flatten()
    df_temp=pd.read_csv(file, names=names)
    all_rows+=len(df_temp)
    dirty_rows+=len(df_temp.loc[(df_temp["lng"]<=0) | (df_temp["lat"]<=0) | (df_temp["speed"]<=0) | (df_temp["speed"]>=120)])
    
print(all_rows)
print(dirty_rows)
print(dirty_rows/all_rows)

notify(f"dirty_rows/all_rows = {dirty_rows/all_rows}")

2545609441
1239008935
0.4867238921432017


TypeError: unsupported operand type(s) for |: 'str' and 'set'

---

In [ ]:
# 这两个日期是需要处理的文件日期开头以及结尾+1
open_day = pd.to_datetime("2020-06-01 00:00:00")
close_day = pd.to_datetime("2020-07-01 00:00:00")

# 将数据进行初步处理
# dest_path是中间结果，暂存到一个文件夹中，之后读取再进行处理
dest_path = "../../data/taxi_after_proc/intermediate/"
if not os.path.exists(dest_path):
    os.mkdir(dest_path)

all_files.sort()
for each_day in tqdm(all_files):
    each_day.sort()

    for each_time in tqdm(each_day):
        records = pd.read_csv(each_time, names=names)
        group = records.groupby("license_number")
        for each_lincense, each_record in group:
            posi = each_record[["lat", "lng", "gps_time", "speed"]]

            # 按gps时间排序，并去重
            posi["gps_time"] = pd.to_datetime(posi["gps_time"])
            posi.sort_values(by="gps_time", inplace=True, ascending=True)
            # posi = posi.drop_duplicates(subset="gps_time", keep="first")
            posi = posi.drop_duplicates(subset=["lat", "lng"], keep="first") # 改成按经纬度去重，尽量保留 speed=0 但是位置有变化的行，之后不再 drop speed=0

            posi = posi.loc[
                # 去除 GPS 0 值
                (posi["lat"] > 0) & (posi["lng"] > 0) &
                # 去除错误日期的记录
                (posi["gps_time"] > open_day) & (posi["gps_time"] < close_day) &
                # 保留速度 0~150 的记录 !这里为了尽量少删，保留速度=0，静止的行由上一步去重来删掉 2022-04-03
                # (posi["speed"] > 0) & 
                (posi["speed"] < 150)]

            # 去除数量过少的记录
            # if len(posi) < 30:
            #     continue

            # 将结果输出到 dest_path 中
            posi.to_pickle(dest_path + each_time[-7:] + each_lincense + ".pkl")
            
notify("Finish intermediate.")

 20%|██        | 2/10 [04:40<18:40, 140.06s/it]

In [ ]:
res_path="../../data/taxi_after_proc/clean202006/"
if not os.path.exists(res_path):
    os.mkdir(res_path)

files = os.listdir(dest_path)
car_set = set()

for file in files:
    car = file[7:-4]
    car_set.add(car)

days = []
for i in range(1, 31):
    days.append(str(i).zfill(2))

for car in tqdm(car_set):
    for day in days:
        if day=="19" or day=="20":
            continue
        df_all = pd.DataFrame()
        for i in range(11):
            try:
                df_all = df_all.append(
                    pd.read_pickle(
                        "../../data/taxi_after_proc/intermediate/06-{}_{}{}.pkl".
                        format(day, i, car)))
            except FileNotFoundError:
                continue
        if df_all.empty:
            continue
        df_all.sort_values(by="gps_time", inplace=True, ascending=True)
        df_all.drop_duplicates(subset=["lat", "lng"], keep="first", inplace=True)
        df_all.drop_duplicates(subset="gps_time", keep="first", inplace=True)
        df_all.to_pickle(
            "../../data/taxi_after_proc/clean202006/06-{}_{}.pkl".format(day, car))
        
notify("Finish 202006.")

  1%|          | 306/37486 [11:46<21:08:08,  2.05s/it]

In [ ]:
NUM_ROWS_ALL=2545609441
num_rows_remain=0
for processed_file in os.listdir(res_path):
    df_temp=pd.read_pickle(os.path.join(res_path, processed_file))
    num_rows_remain+=len(df_temp)
    
print(NUM_ROWS_ALL)
print(num_rows_remain)
print(num_rows_remain/NUM_ROWS_ALL)

notify(f"remaining_rows/all_rows = {num_rows_remain/NUM_ROWS_ALL}")